In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Omitimos la importación de TensorFlow, Keras y scikeras
# Omitimos la función crear_modelo_mlp

# Diccionario de modelos (solo modelos ligeros)
modelos = {
    "LogisticRegression": LogisticRegression(
        penalty='l2', C=1.0, solver='saga', max_iter=2000, random_state=42
    ),
    "DecisionTree": DecisionTreeClassifier(
        max_depth=5, min_samples_split=10, min_samples_leaf=5, random_state=42
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=100, max_depth=8, min_samples_split=10, min_samples_leaf=5,
        n_jobs=-1, random_state=42
    ),
    "XGBoost": XGBClassifier(
        n_estimators=100, max_depth=6, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8, eval_metric='logloss',
        use_label_encoder=False, random_state=42
    ),
}

### Función de evaluación

In [2]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, classification_report
)
import time

# Lista global donde se acumulan los resultados
resultados_modelos = []

def evaluar_modelo(nombre_modelo, modelo, X_train, X_test, y_train, y_test, escenario=""):
    inicio = time.time()

    modelo.fit(X_train, y_train)

    if hasattr(modelo, "predict_proba"):
        y_proba = modelo.predict_proba(X_test)[:, 1]
        y_pred = (y_proba >= 0.5).astype(int)
    elif hasattr(modelo, "decision_function"):
        y_proba = modelo.decision_function(X_test)
        y_pred = (y_proba >= 0).astype(int)
    else:
        y_pred = modelo.predict(X_test)
        y_proba = None

    fin = time.time()
    duracion = fin - inicio

    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    spec = tn / (tn + fp)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None

    # Imprimir resultados
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy     : {acc:.3f}")
    print(f"Recall       : {rec:.3f}")
    print(f"Precision    : {prec:.3f}")
    print(f"F1-score     : {f1:.3f}")
    print(f"Specificity  : {spec:.3f}")
    print(f"AUC-ROC      : {auc:.3f}" if auc else "AUC-ROC: N/A")
    print(f"Tiempo de ejecución: {duracion:.2f} segundos")

    # Guardar en la lista
    resultados_modelos.append({
        "Escenario": escenario,
        "Modelo": nombre_modelo,
        "Accuracy": acc,
        "Recall": rec,
        "Precision": prec,
        "F1-score": f1,
        "Specificity": spec,
        "AUC-ROC": auc,
        "Tiempo (s)": duracion
    })


# Función para guardar todo en CSV
def guardar_resultados_csv(ruta="resultados_modelos.csv"):
    df_resultados = pd.DataFrame(resultados_modelos)
    df_resultados.to_csv(ruta, index=False)
    print(f"Resultados guardados en {ruta}")


### Escenario 1 – Dataset limpio completo

In [3]:
import time

df1 = pd.read_csv("/kaggle/input/data-csv/Hipertension_Arterial_Mexico_limpio.csv")
y1 = df1["riesgo_hipertension"]

print("Escenario 1 – Dataset completo")

for nombre, modelo in modelos.items():
    print("=" * 40)
    print(f"--- {nombre} ---")

    X1 = df1.drop(columns=["riesgo_hipertension", "FOLIO_I"]).copy()

    # One-Hot Encoding para 'sueno_horas' solo si el modelo lo requiere
    if "sueno_horas" in X1.columns and nombre == "LogisticRegression":
        X1 = pd.get_dummies(X1, columns=["sueno_horas"], drop_first=True)

    # Escalar solo si el modelo lo requiere
    if nombre == "LogisticRegression":
        scaler = StandardScaler()
        X1 = scaler.fit_transform(X1)

    # Entrenamiento y evaluación
    X1_train, X1_test, y1_train, y1_test = train_test_split(
        X1, y1, test_size=0.2, stratify=y1, random_state=42
    )

    evaluar_modelo(nombre, modelo, X1_train, X1_test, y1_train, y1_test, escenario="Escenario 1")

# Guardar resultados acumulados en CSV general
guardar_resultados_csv("resultados_todos_escenarios.csv")


Escenario 1 – Dataset completo
--- LogisticRegression ---
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.63      0.66       309
           1       0.81      0.85      0.83       563

    accuracy                           0.77       872
   macro avg       0.75      0.74      0.74       872
weighted avg       0.77      0.77      0.77       872

Accuracy     : 0.771
Recall       : 0.849
Precision    : 0.806
F1-score     : 0.827
Specificity  : 0.628
AUC-ROC      : 0.814
Tiempo de ejecución: 2.48 segundos
--- DecisionTree ---
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       309
           1       0.98      0.98      0.98       563

    accuracy                           0.98       872
   macro avg       0.97      0.97      0.97       872
weighted avg       0.98      0.98      0.98       872

Accuracy     : 0.976
Recall       : 0.980
Precision    : 0.9

### Escenario 2 – Dataset con selección FSS

In [4]:
import time

df2 = pd.read_csv("/kaggle/input/data-csv/variables_seleccionadas.csv")
y2 = df2["riesgo_hipertension"]

print("Escenario 2 – Dataset con FSS")

for nombre, modelo in modelos.items():
    print("=" * 40)
    print(f"--- {nombre} ---")

    X2 = df2.drop(columns=["riesgo_hipertension", "FOLIO_I"]).copy()

    # One-Hot Encoding para 'sueno_horas' solo si el modelo lo requiere
    if "sueno_horas" in X2.columns and nombre == "LogisticRegression":
        X2 = pd.get_dummies(X2, columns=["sueno_horas"], drop_first=True)

    # Escalar solo si el modelo lo requiere
    if nombre == "LogisticRegression":
        scaler = StandardScaler()
        X2 = scaler.fit_transform(X2)

    # Entrenamiento y evaluación
    X2_train, X2_test, y2_train, y2_test = train_test_split(
        X2, y2, test_size=0.2, stratify=y2, random_state=42
    )

    evaluar_modelo(nombre, modelo, X2_train, X2_test, y2_train, y2_test, escenario="Escenario 2")

# Guardar resultados acumulados en CSV general
guardar_resultados_csv("resultados_todos_escenarios.csv")


Escenario 2 – Dataset con FSS
--- LogisticRegression ---
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.64      0.69       309
           1       0.82      0.87      0.84       563

    accuracy                           0.79       872
   macro avg       0.78      0.76      0.76       872
weighted avg       0.79      0.79      0.79       872

Accuracy     : 0.791
Recall       : 0.874
Precision    : 0.816
F1-score     : 0.844
Specificity  : 0.641
AUC-ROC      : 0.815
Tiempo de ejecución: 0.99 segundos
--- DecisionTree ---
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       309
           1       0.99      0.98      0.98       563

    accuracy                           0.98       872
   macro avg       0.98      0.98      0.98       872
weighted avg       0.98      0.98      0.98       872

Accuracy     : 0.978
Recall       : 0.980
Precision    : 0.98

### Escenario 3 – Wrapper (RFE) con dataset completo

In [5]:
import time

print("Escenario 3 – Selección por Wrapper (SFS con k=15)")
df3 = pd.read_csv("/kaggle/input/data-csv/Hipertension_Arterial_Mexico_limpio.csv")

X3_base = df3.drop(columns=["riesgo_hipertension", "FOLIO_I"])
y3 = df3["riesgo_hipertension"]

for nombre, modelo in modelos.items():
    if nombre == "XGBoost":
        print("XGBoost omitido de SFS por incompatibilidad actual con mlxtend y scikit-learn.")
        continue

    print("=" * 40)
    print(f"--- {nombre} con SFS (15 vars) ---")

    X3 = X3_base.copy()

    if nombre == "LogisticRegression":
        if "sueno_horas" in X3.columns:
            X3 = pd.get_dummies(X3, columns=["sueno_horas"], drop_first=True)
        scaler = StandardScaler()
        X3 = scaler.fit_transform(X3)
        feature_names = pd.get_dummies(X3_base, columns=["sueno_horas"], drop_first=True).columns
    else:
        feature_names = X3.columns
        X3 = X3.values

    inicio = time.time()
    sfs = SFS(modelo,
              k_features=15,
              forward=True,
              floating=False,
              scoring='roc_auc',
              cv=5,
              n_jobs=1)
    sfs = sfs.fit(X3, y3)
    selected_idx = list(sfs.k_feature_idx_)
    X3_selected = X3[:, selected_idx]

    X3_train, X3_test, y3_train, y3_test = train_test_split(
        X3_selected, y3, test_size=0.2, stratify=y3, random_state=42)

    evaluar_modelo(nombre, modelo, X3_train, X3_test, y3_train, y3_test, escenario="Escenario 3")

    fin = time.time()
    print(f"Tiempo total del proceso (selección + evaluación): {fin - inicio:.2f} segundos")
    print("Variables seleccionadas:", list(feature_names[selected_idx]))

# Guardar los resultados de este escenario (opcional si ya lo haces al final del script)
guardar_resultados_csv("resultados_todos_escenarios.csv")


Escenario 3 – Selección por Wrapper (SFS con k=15)
--- LogisticRegression con SFS (15 vars) ---
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.63      0.67       309
           1       0.81      0.85      0.83       563

    accuracy                           0.77       872
   macro avg       0.75      0.74      0.75       872
weighted avg       0.77      0.77      0.77       872

Accuracy     : 0.774
Recall       : 0.851
Precision    : 0.809
F1-score     : 0.829
Specificity  : 0.634
AUC-ROC      : 0.818
Tiempo de ejecución: 0.93 segundos
Tiempo total del proceso (selección + evaluación): 467.20 segundos
Variables seleccionadas: ['sexo', 'edad', 'concentracion_hemoglobina', 'valor_acido_urico', 'valor_colesterol_ldl', 'valor_trigliceridos', 'valor_transferrina', 'distancia_rodilla_talon', 'circunferencia_de_la_pantorrilla', 'tension_arterial', 'actividad_total', 'peso_corregido', 'estatura_corregida', 'imc', 'sueno_horas_4']

In [6]:
import time

print("Escenario 4 – Selección automática con Wrapper (SFS best)")
df4 = pd.read_csv("/kaggle/input/data-csv/Hipertension_Arterial_Mexico_limpio.csv")

X4_base = df4.drop(columns=["riesgo_hipertension", "FOLIO_I"])
y4 = df4["riesgo_hipertension"]

for nombre, modelo in modelos.items():
    if nombre == "XGBoost":
        print("XGBoost omitido de SFS por incompatibilidad actual con mlxtend y scikit-learn.")
        continue

    print("=" * 50)
    print(f"--- {nombre} con SFS automático ---")

    X4 = X4_base.copy()

    if nombre == "LogisticRegression":
        if "sueno_horas" in X4.columns:
            X4 = pd.get_dummies(X4, columns=["sueno_horas"], drop_first=True)
        scaler = StandardScaler()
        X4 = scaler.fit_transform(X4)
        feature_names = pd.get_dummies(X4_base, columns=["sueno_horas"], drop_first=True).columns
    else:
        feature_names = X4.columns
        X4 = X4.values

    inicio = time.time()
    sfs = SFS(modelo,
              k_features='best',
              forward=True,
              floating=False,
              scoring='roc_auc',
              cv=5,
              n_jobs=1)
    sfs = sfs.fit(X4, y4)
    selected_idx = list(sfs.k_feature_idx_)
    X4_selected = X4[:, selected_idx]

    X4_train, X4_test, y4_train, y4_test = train_test_split(
        X4_selected, y4, test_size=0.2, stratify=y4, random_state=42)

    evaluar_modelo(nombre, modelo, X4_train, X4_test, y4_train, y4_test, escenario="Escenario 4")

    fin = time.time()
    print(f"Tiempo total del proceso (selección + evaluación): {fin - inicio:.2f} segundos")
    print("Número óptimo de variables:", len(selected_idx))
    print("Variables seleccionadas:", list(feature_names[selected_idx]))

# Guardar resultados acumulados
guardar_resultados_csv("resultados_todos_escenarios.csv")


Escenario 4 – Selección automática con Wrapper (SFS best)
--- LogisticRegression con SFS automático ---
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.64      0.67       309
           1       0.81      0.86      0.83       563

    accuracy                           0.78       872
   macro avg       0.76      0.75      0.75       872
weighted avg       0.78      0.78      0.78       872

Accuracy     : 0.781
Recall       : 0.858
Precision    : 0.813
F1-score     : 0.835
Specificity  : 0.641
AUC-ROC      : 0.819
Tiempo de ejecución: 0.98 segundos
Tiempo total del proceso (selección + evaluación): 1464.68 segundos
Número óptimo de variables: 19
Variables seleccionadas: ['sexo', 'edad', 'concentracion_hemoglobina', 'valor_acido_urico', 'valor_colesterol_hdl', 'valor_colesterol_ldl', 'valor_trigliceridos', 'valor_transferrina', 'valor_vitamina_d', 'distancia_rodilla_talon', 'circunferencia_de_la_pantorrilla', 'tension_arterial'